In [98]:
import pandas as pd
import numpy as np

In [102]:
def patient_class(x):
    if " IP" in x or "Inpatient" in x:
        return 'inpatient'
    elif " OP" in x or "Outpatient" in x:
        return 'outpatient'
    else:
        return 'both'

def payer_category(x):
    payers = {
        'Gross Charges': 'gross',
        'De-identified Minimum IP Charge': 'min',
        'De-identified Minimum OP Charge': 'min',
        'De-identified Maximum Charge IP': 'max',
        'De-identified Maximum Charge OP': 'max',
        'SELF PAY BEFORE FINANCIAL ASSISTANCE': 'cash'
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

def code_prefix(x):
    x = str(x)
    prefixes = {
        'HCPCS': 'hcpcs_cpt',
        'CPT®': 'hcpcs_cpt',
        'Custom': 'custom'
    }

    if " " in x:
        prefix = x.split(" ")[0]
        try:
            return prefixes[prefix]
        except KeyError:
            print(x)
    else:
        return 'none'

def zero_pad(code, code_type):
    if code_type == 'drg':
        return str(code).zfill(3)
    else:
        return code

In [109]:
folder = ".\\input_files\\"
file = '58-0572465_Childrens_Healthcare_of_ATL_at_Scottish_Rite_standardcharges.xlsx'
ein = file.split("_")[0]

ccn = {
    '58-0572465': '113301',
    '58-0572412': '113300',
    '20-4144787': '113300',
}

In [88]:
df = pd.read_excel(folder + file, skiprows=1, dtype={'Code': str, 'Procedure': str, 'NDC': str, 'Rev Code': str})

In [89]:
df.rename(columns={
    'Procedure': 'internal_code',
    'Code Type': 'code_type',
    'Code': 'code_orig',
    'NDC': 'ndc',
    'Rev Code': 'rev_desc',
    'Procedure Description': 'description',
}, inplace=True)

In [90]:
cols = df.columns.tolist()
id_vars = cols[:cols.index('Gross Charges')]
value_vars = cols[cols.index('Gross Charges'):]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_orig', value_name='rate')


In [91]:
df['patient_class'] = df['payer_orig'].apply(patient_class)
df['payer_category'] = df['payer_orig'].apply(payer_category)

In [93]:
df['rev_code'] = df['rev_desc'].apply(lambda x: str(x).split(' - ')[0])

In [95]:
df.columns

Index(['internal_code', 'code_type', 'code_orig', 'ndc', 'rev_desc',
       'description', 'Quantity', 'payer_orig', 'rate', 'patient_class',
       'payer_category', 'rev_code'],
      dtype='object')

In [105]:
df['code_type'] = df['code_type'].str.replace('Champus DRG - v35', 'drg')

zero_pad_func = np.vectorize(zero_pad)
df['code'] = zero_pad_func(df['code_orig'], df['code_type'])
df['code_prefix'] = df['code'].apply(code_prefix)

df['code'] = df['code'].apply(lambda x: str(x).split()[-1] if ' ' in str(x) else x)

In [108]:
df['internal_code'].fillna('na', inplace=True)
df['code'].fillna('na', inplace=True)
df['rev_code'].fillna('na', inplace=True)

In [110]:
df['payer_name'] = df['payer_orig']
df['code_type'] = df['code_type'].str.lower()

df['filename'] = file
df['hospital_ein'] = ein
df['hospital_ccn'] = ccn[ein]
df['file_last_updated'] = '2023-01-01'

In [111]:
df.to_csv('ein' + '.csv', index=False)

,internal_code,code_type,code_orig,ndc,rev_desc,description,Quantity,payer_orig,rate,patient_class,payer_category,rev_code,code,code_prefix,payer_name,filename,hospital_ein,hospital_ccn
0,25001009,eap,HCPCS Q9966,NaN,0250 - PHARMACY - GENERAL,"Locm Omnipaque 240, per Ml",1,Gross Charges,14.0,both,gross,0250,Q9966,hcpcs_cpt,Gross Charges,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
1,27009888,eap,HCPCS C1729,NaN,0270 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,Fuhrman Pleural Drain 6.0 (85948),1,Gross Charges,1540.0,both,gross,0270,C1729,hcpcs_cpt,Gross Charges,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
2,27009889,eap,HCPCS C1751,NaN,0270 - MEDICAL/SURGICAL SUPPLIES AND DEVICES -...,Cath Triple Lumen Uvc 5fr,1,Gross Charges,63.0,both,gross,0270,C1751,hcpcs_cpt,Gross Charges,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
3,36001008,eap,CPT® 61708,NaN,0360 - OPERATING ROOM SERVICES - GENERAL,Aneurysm Reapir in Brain,1,Gross Charges,10877.0,both,gross,0360,61708,hcpcs_cpt,Gross Charges,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
4,36005074,eap,CPT® 36160,NaN,0360 - OPERATING ROOM SERVICES - GENERAL,Insertion of Needle or Catheter Into Aorta,1,Gross Charges,771.0,both,gross,0360,36160,hcpcs_cpt,Gross Charges,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547023,na,ed,99285,NaN,0450 - EMERGENCY ROOM - GENERAL,ER LEVEL 5,NaN,VERACITY BENEFITS: CHILDREN'S,NaN,both,payer,0450,99285,none,VERACITY BENEFITS: CHILDREN'S,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
1547024,na,ed,99284,NaN,0450 - EMERGENCY ROOM - GENERAL,ER LEVEL 4,NaN,VERACITY BENEFITS: CHILDREN'S,NaN,both,payer,0450,99284,none,VERACITY BENEFITS: CHILDREN'S,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
1547025,na,ed,99283,NaN,0450 - EMERGENCY ROOM - GENERAL,ER LEVEL 3,NaN,VERACITY BENEFITS: CHILDREN'S,NaN,both,payer,0450,99283,none,VERACITY BENEFITS: CHILDREN'S,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
1547026,na,ed,99282,NaN,0450 - EMERGENCY ROOM - GENERAL,ER LEVEL 2,NaN,VERACITY BENEFITS: CHILDREN'S,NaN,both,payer,0450,99282,none,VERACITY BENEFITS: CHILDREN'S,58-0572465_Childrens_Healthcare_of_ATL_at_Scot...,58-0572465,113301
